In [1]:
import pandas as pd
import re
from collections import defaultdict
import math
import numpy as np

In [2]:
def preprocess(text):
    """Очистка текста и токенизация"""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Удаляем пунктуацию
    tokens = text.split()
    return tokens

class SimpleWord2Vec:
    def __init__(self, window_size=2):
        self.window_size = window_size
        self.word_counts = defaultdict(int)
        self.co_occur = defaultdict(lambda: defaultdict(int))
        self.vectors = {}
    
    def train(self, texts):
        """Обучение на коллекции текстов"""
        # Собираем статистику
        for text in texts:
            tokens = preprocess(text)
            for i, word in enumerate(tokens):
                self.word_counts[word] += 1
                start = max(0, i - self.window_size)
                end = min(len(tokens), i + self.window_size + 1)
                for j in range(start, end):
                    if j != i:
                        self.co_occur[word][tokens[j]] += 1
        
        # Создание векторов (PMI)
        total_word_pairs = sum(sum(inner.values()) for inner in self.co_occur.values())
        for word in self.co_occur:
            vector = []
            word_total = sum(self.co_occur[word].values())
            for context_word in self.word_counts:
                # Правильный расчет PMI
                joint = self.co_occur[word].get(context_word, 0)
                p_word = self.word_counts[word] / total_word_pairs
                p_context = self.word_counts[context_word] / total_word_pairs
                p_joint = joint / total_word_pairs
                
                if p_joint > 0:
                    pmi = math.log(p_joint / (p_word * p_context))
                else:
                    pmi = 0
                
                vector.append(pmi)
            self.vectors[word] = np.array(vector)

class SemanticSearch:
    def __init__(self):
        self.model = SimpleWord2Vec()
        self.texts = []
    
    def index(self, documents):
        """Индексация документов"""
        self.texts = documents
        self.model.train(documents)
    
    def cosine_sim(self, vec_a, vec_b):
        """Косинусная мера между векторами"""
        if np.linalg.norm(vec_a) == 0 or np.linalg.norm(vec_b) == 0:
            return 0
        return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
    
    def search(self, query, threshold=0.6):
        """Поиск совпадений"""
        results = []
        query_tokens = preprocess(query)
        
        for doc_id, text in enumerate(self.texts):
            tokens = preprocess(text)
            if not tokens:
                continue
                
            # Вектор документа (среднее векторов слов)
            doc_vector = sum(self.model.vectors.get(word, np.zeros(len(self.model.word_counts))) 
                         for word in tokens) / len(tokens)
            
            for phrase in query_tokens:
                phrase_vec = self.model.vectors.get(phrase, np.zeros(len(self.model.word_counts)))
                similarity = self.cosine_sim(phrase_vec, doc_vector)
                
                if similarity > threshold:
                    # Поиск точной позиции
                    for i, word in enumerate(tokens):
                        word_sim = self.cosine_sim(
                            self.model.vectors.get(word, np.zeros(len(self.model.word_counts))),
                            phrase_vec
                        )
                        if word_sim > threshold:
                            start_pos = sum(len(tokens[j]) + 1 for j in range(i))
                            end_pos = start_pos + len(word)
                            results.append({
                                "text": text,
                                "target": phrase,
                                "start_pos": start_pos,
                                "end_pos": end_pos,
                                "score": similarity,
                                "doc_id": doc_id
                            })
        return results

In [3]:
df = pd.read_csv('../Data/NeuroEmotions_data.csv')

search_engine = SemanticSearch()
search_engine.index(df['doc_text_clean'])

In [6]:
results = search_engine.search("склейка", threshold=0.3)

for res in results:
    print(f"Документ {res['doc_id']}:")
    print(f"Текст: {res['text']}")
    print(f"Найдено: '{res['text'][res['start_pos']:res['end_pos']]}'")
    print(f"Позиция: {res['start_pos']}-{res['end_pos']}")
    print(f"Схожесть: {res['score']:.2f}")
    print("-" * 50)

## Описание работы системы семантического поиска:

**1. Препроцессинг текста**

Система начинает с очистки входного текста: переводит все символы в нижний регистр и удаляет пунктуацию, оставляя только слова и пробелы. Затем текст разбивается на отдельные слова (токены). Например, фраза "Мама мыла раму!" преобразуется в список ["мама", "мыла", "раму"].

**2. Обучение модели Word2Vec**

Система создает семантические векторные представления слов по следующему алгоритму:

Собирает статистику совместной встречаемости слов в пределах заданного окна (по умолчанию 2 слова слева и справа)

Для каждого слова рассчитывает PMI (Pointwise Mutual Information) - меру семантической связи между словами

Строит векторное пространство, где каждое слово представлено вектором его PMI-значений со всеми другими словами в словаре

**3. Индексация документов**

При добавлении документов система:

Сохраняет оригинальные тексты

Для каждого документа вычисляет усредненный вектор как среднее векторов всех слов в документе

Создает поисковый индекс, позволяющий быстро находить документы по их векторным представлениям

**4. Поиск и ранжирование**

При обработке поискового запроса система:

Разбивает запрос на слова и находит их векторные представления

Вычисляет косинусную схожесть между вектором запроса и векторами документов

Отбирает документы с показателем схожести выше заданного порога (по умолчанию 0.6)

**5. Определение позиций**

Для найденных документов система дополнительно:

Ищет точные позиции слов запроса в тексте

При отсутствии точных совпадений находит семантически близкие слова

Возвращает позиции начала и конца найденных фрагментов

**6. Особенности работы**

Полностью на Python без внешних зависимостей (кроме numpy)

Автоматически обрабатывает слова, отсутствующие в словаре

Учитывает семантическую близость, а не только точные совпадения

Позволяет настраивать чувствительность поиска через порог схожести